In [2]:
import json
import requests
import pandas as pd # module to convert an address into latitude and longitude values
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim #Module for converting the address into latitude and longitude value
from IPython.display import display
from bs4 import BeautifulSoup #Helps us to get the HTML structure of the page



In [3]:
Address = '75 Thorncliffe Park Dr, Toronto, Ontario'

geolocation = Nominatim(user_agent="foursquare_agent")
location = geolocation.geocode(Address)
latitude = location.latitude
longitude = location.longitude

query = "Food"
limit = 10
radius = 500
print(latitude, longitude)

43.7049593 -79.3419739


In [4]:
#Call to the api using the stored variables
url = "https://api.foursquare.com/v3/places/search?query={}&ll={},{}&radius={}".format(query,latitude,longitude, radius)

headers = {
    "Accept": "application/json",
    "Authorization": "fsq30W5Rz0P5HJTL4F1jGkXsg37CiGhYOejnEqQCPdKwfUI="
}

#Response from the call
response = requests.request("GET", url, headers=headers)

#Convert the data to JSON
json_data = json.loads(response.text)

#Get the result and store it in list
listt = json_data["results"]

#For every hash, find the key "name"
for restaurant in listt: 
    print(restaurant["name"])


mymap = folium.Map(location=[43.70, -79.34], zoom_start=0)
mymap.save('mymap.html')



Bamiyan Kabob
A&W Restaurant
Pasha Authentic Turkish Cuisine
Faley Restaurant
Afghan Cuisine
Anatolian's Turkish Halal Restaurant
Popeye's Louisiana Kitchen
Hero Certified Burgers
Swiss Chalet Rotisserie & Grill
Capital Sweet Smousa


In [5]:

torontodata = pd.read_csv('Neighbourhoods_data.csv')
#list(torontodata)
#torontodata.head(1)

#Display all the data from the coloumn LATITUDE
#torontodata.loc[:, ['AREA_ID']]

In [6]:
#Create a list to store postal codes and neighborhood names
postalCode=[]
neighborhood=[]

#Website to extract the data from
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#Get the website and change it into HTML format
soup = BeautifulSoup(source,'html.parser')

table = soup.find().tbody

#find all rows that contain the body tag
row = table.findAll('b')

#for each column in the the row, find the body text(postal code)
for column in row:
    #Add it to the postal code list
    postalCode.append(column.get_text())
    #print(column.get_text()) #Check what is printing out

#find all rows that contain the body tag
row = table.findAll('span')

#for each column in the the row, find the span text(neighborhood)
for column in row:
    #Add it to the neighborhood list
    neighborhood.append(column.get_text())
    #print(column.get_text())

#Add postal code and neighborhood to the dataframe
df = pd.DataFrame({'PostalCode':postalCode,'Neighborhood':neighborhood})
print(df)





    PostalCode                                       Neighborhood
0          M1A                                       Not assigned
1          M2A                                       Not assigned
2          M3A                              North York(Parkwoods)
3          M4A                       North York(Victoria Village)
4          M5A       Downtown Toronto(Regent Park / Harbourfront)
..         ...                                                ...
175        M5Z                                       Not assigned
176        M6Z                                       Not assigned
177        M7Z                                       Not assigned
178        M8Z  Etobicoke(Mimico NW / The Queensway West / Sou...
179        M9Z                                       Not assigned

[180 rows x 2 columns]


In [7]:
#Create a list to store postal codes and neighborhood names
postalCode=[]
neighborhood=[]
district = []

#Website to extract the data from
url = "https://www.zipcodesonline.com/2020/06/postal-code-of-toronto-in-2020.html"

#Read the html tables into a list
scrapper = pd.read_html(url,skiprows=2)

#Store the date into the list 
neighborhood = scrapper[0][1]
postalCode = scrapper[0][2]
district =  scrapper[0][3]

#Check if data is right
#print(neighborhood)
#print(postalCode)
#print(district)

#Create a data fram and store the scraped data
postalCodeWithNeighborhood = pd.DataFrame({'Postal Code':postalCode,'Neighborhood':neighborhood, 'District':district})
postalCodeWithNeighborhood.columns = ['Postal Code', 'Neighborhood', 'District']

#postalCodeWithNeighborhood.index += 1 #start the index from 1 and not zero

postalCodeWithNeighborhood.sort_values(by=['Postal Code'])
#Display the postal code through column postal code
#postalCodeWithNeighborhood.loc[:, "Postal Code"]

#postalCodeWithNeighborhood.head()


,Postal Code,Neighborhood,District
141,M1B,Rouge,Scarborough
97,M1B,Malvern,Scarborough
65,M1C,Highland Creek,Scarborough
140,M1C,Rouge Hill,Scarborough
128,M1C,Port Union,Scarborough
...,...,...,...
111,M9V,Mount Olive,Etobicoke
3,M9V,Albion Gardens,Etobicoke
71,M9V,Humbergate,Etobicoke
117,M9W,Northwest,Etobicoke


In [8]:
#Postal Code and Neighborhood Name Data

# reading tables with pandas from url
tables = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=876823784')

# number of tables on website
print(f'number of tables: {len(tables)}')

# show the firsts rows of tables
for i in range(len(tables)):
  print(tables[i].head())


  # new dataframe with first table
df = tables[0]
df.head()

# create a new dataframe drop Not assigned in Borough
postalCodeWithNeighborhoodName = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
postalCodeWithNeighborhoodName.rename(columns={'Postcode':'Postal Code'}, inplace=True)
postalCodeWithNeighborhoodName.to_csv("data.csv")

number of tables: 3
  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
  .mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output .navbar-brackets::before{margin-right:-0.125em;content:"[ "}.mw-parser-output .navbar-brackets::after{margin-left:-0.125em;content:" ]"}.mw-parser-output .navbar li{word-spacing:-0.125em}.mw-parser-output .navbar a>span,.mw-parser-output .navbar a>abbr{text-decoration:inherit}.mw-parser-output .navbar-mini abbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-o

In [9]:
#Postal Code, Longitude, Latitude Data

postalCodeLongLat = pd.read_csv('PostalCode_Longitute_Latitude.csv')
#Constructing Hash

postalCodeLongLat.rename(columns={'Postal Code':'Postal Code'}, inplace=True)


postalCodeLongLat.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#Join the data only if the postal code key matches
print(postalCodeWithNeighborhoodName.columns)
print(postalCodeLongLat.columns)

mergeData = postalCodeWithNeighborhoodName.set_index("Postal Code").join(postalCodeLongLat.set_index('Postal Code'), on= 'Postal Code')

#mergeData.rename(columns={'Postal Code':'Postal Code', 'Postal Code':'Postal Code', 'Neighbourhood': 'Neighbourhood', 'District': 'District', 'Latitude':'Latitude', 'Longitude':'Longitude'}, inplace=True)

#Sort the data in terms of postal code (Ascending Order)
mergeData.sort_values(['Postal Code'], inplace=True)
mergeData.to_csv(r'temp.csv')


Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')
Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')


In [11]:
#Income Data

#Load CSV File
df = pd.read_csv('neighbourhood-profiles-2016.csv')

#Drop first 3 columns
df.drop(df.columns[[0,1,3]], inplace=True, axis = 1)


df.set_index('Topic', inplace = True)
NeighborhoodIncomes = df.transpose()

NeighborhoodIncomes.reset_index(inplace=True) #reset index of dataframe
for column in NeighborhoodIncomes.columns:
    try:
        column.strip()
    except:
        print(column)

NeighborhoodIncomes.rename(columns={'index': 'Neighbourhood'},inplace = True) #rename column header
NeighborhoodIncomes.drop(NeighborhoodIncomes.columns.difference(['Income of households in 2015', 'Neighbourhood']), axis=1, inplace=True)
NeighborhoodIncomes.columns = NeighborhoodIncomes.iloc[0] #these next 3 lines remove top row
NeighborhoodIncomes = NeighborhoodIncomes.reindex(NeighborhoodIncomes.index.drop(0)).reset_index(drop=True)
NeighborhoodIncomes.columns.name = None 
NeighborhoodIncomes.rename(columns={'Characteristic' : 'Neighbourhood'}, inplace=True)

neighbourhoodCol = NeighborhoodIncomes[['Neighbourhood']].copy()

#Get the column data only
NeighborhoodIncomes = NeighborhoodIncomes.loc[:,'Total - Household total income groups in 2015 for private households - 100% data':'    $200,000 and over']

finalIncome = neighbourhoodCol.join(NeighborhoodIncomes)

finalIncome.to_csv("income.csv")
#NeighborhoodIncomes.to_csv("income.csv")

In [12]:
#Crime Rate Data

crimeRateData = pd.read_csv('Neighbourhood_Crime_Rates_(Boundary_File)_.csv')
crimeRateData.drop(crimeRateData.columns[[0]], inplace=True, axis = 1)
crimeRateData.rename(columns={'Neighbourhood':'Neighbourhood'}, inplace=True)
crimeRateData.head()

crimeRateData.to_csv("CrimeRateData.csv")
crimeRateData



,Neighbourhood,Hood_ID,Population,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,Assault_AVG,...,TheftOver_2015,TheftOver_2016,TheftOver_2017,TheftOver_2018,TheftOver_2019,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2019,Shape__Area,Shape__Length
0,Yonge-St.Clair,97,12528,20,29,39,27,34,37,31.0,...,5,8,0,3,6,4.3,1.00,47.9,1.161315e+06,5873.270582
1,York University Heights,27,27593,271,296,361,344,357,370,333.2,...,46,37,39,38,28,36.3,-0.26,101.5,1.324666e+07,18504.777326
2,Lansing-Westgate,38,16164,44,80,68,85,75,72,70.7,...,5,5,11,6,11,7.0,0.83,68.1,5.346186e+06,11112.109625
3,Yorkdale-Glen Park,31,14804,106,136,174,161,175,209,160.2,...,14,26,23,20,29,22.5,0.45,195.9,6.038326e+06,10079.426920
4,Stonegate-Queensway,16,25051,88,71,76,95,87,82,83.2,...,8,4,6,7,4,6.0,-0.43,16.0,7.946202e+06,11853.189878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Milliken,130,26572,66,92,88,80,78,99,83.8,...,14,10,19,18,22,16.5,0.22,82.8,9.477859e+06,16781.477343
136,Pleasant View,46,15818,34,50,58,53,37,44,46.0,...,5,3,4,5,4,3.8,-0.20,25.3,2.967493e+06,7944.829848
137,Wychwood,94,14349,54,76,86,55,76,74,70.2,...,1,4,4,2,0,2.3,-1.00,0.0,1.682111e+06,5435.021879
138,Leaside-Bennington,56,16828,38,27,33,35,33,31,32.8,...,5,2,4,4,7,5.2,0.75,41.6,4.784766e+06,14836.618578


In [13]:
#Neighborhood Data

neighborhoodData = pd.read_csv('Neighbourhoods - 4326.csv')
neighborhoodData.drop(neighborhoodData.columns[[0]], inplace=True, axis = 1)
neighborhoodData.rename(columns={'AREA_DESC':'Neighbourhood'}, inplace=True)
neighborhoodData.head()

neighborhoodData.to_csv("NeighborhoodData.csv")


In [14]:
#Combine Crime Rate, Neighborhood, and Income

print(neighborhoodData.columns)
print(crimeRateData.columns)
print(finalIncome.columns)

#Add Income to final data
firstDataset = neighborhoodData.set_index('Neighbourhood').join(finalIncome.set_index('Neighbourhood'), on='Neighbourhood')

#secondDataset = firstDataset.set_index('Neighbourhood').join(finalIncome.set_index('Neighbourhood'), on='Neighbourhood')

#firstDataset.to_csv("final.csv")

#secondDataset.to_csv("final.csv")

#finalData.replace("", "NaN", inplace=True)
#finalData.dropna(how='all', axis=1, inplace=True)

#finalData2 = temp.set_index('Neighbourhood').join(mergeData.set_index('Neighborhood'))



Index(['Neighbourhood', 'AREA_ID', 'AREA_LONG_CODE', 'AREA_NAME',
       'AREA_SHORT_CODE', 'CLASSIFICATION', 'CLASSIFICATION_CODE', 'LATITUDE',
       'LONGITUDE', 'OBJECTID', 'PARENT_AREA_ID', 'Shape__Area',
       'Shape__Length', 'X', 'Y', '_id', 'geometry'],
      dtype='object')
Index(['Neighbourhood', 'Hood_ID', 'Population', 'Assault_2014',
       'Assault_2015', 'Assault_2016', 'Assault_2017', 'Assault_2018',
       'Assault_2019', 'Assault_AVG', 'Assault_CHG', 'Assault_Rate_2019',
       'AutoTheft_2014', 'AutoTheft_2015', 'AutoTheft_2016', 'AutoTheft_2017',
       'AutoTheft_2018', 'AutoTheft_2019', 'AutoTheft_AVG', 'AutoTheft_CHG',
       'AutoTheft_Rate_2019', 'BreakandEnter_2014', 'BreakandEnter_2015',
       'BreakandEnter_2016', 'BreakandEnter_2017', 'BreakandEnter_2018',
       'BreakandEnter_2019', 'BreakandEnter_AVG', 'BreakandEnter_CHG',
       'BreakandEnter_Rate_2019', 'Homicide_2014', 'Homicide_2015',
       'Homicide_2016', 'Homicide_2017', 'Homicide_2018', 'Hom

In [15]:
#Get latitude and Longitude from neighborhoods 

toronto_gdf = gpd.read_file(r'NeighbourhoodsData.geojson') # Read GEOJSON file to a Geopandas dataframe
toronto_gdf.head() # Display initial dataframe to see what outcome is

,AREA_ATTR_ID,AREA_DESC,AREA_ID,AREA_LONG_CODE,AREA_NAME,AREA_SHORT_CODE,CLASSIFICATION,CLASSIFICATION_CODE,LATITUDE,LONGITUDE,OBJECTID,PARENT_AREA_ID,Shape__Area,Shape__Length,X,Y,_id,geometry
0,26005521,Casa Loma (96),2480141,096,Casa Loma (96),096,None,None,None,None,17545105,None,3.678385e+06,8214.176485,None,None,1,"POLYGON ((-79.41469 43.67391, -79.41485 43.674..."
1,26005520,Annex (95),2480140,095,Annex (95),095,None,None,None,None,17545121,None,5.337192e+06,10513.883143,None,None,2,"POLYGON ((-79.39414 43.66872, -79.39588 43.668..."
2,26005519,Caledonia-Fairbank (109),2480139,109,Caledonia-Fairbank (109),109,None,None,None,None,17545137,None,2.955857e+06,6849.911724,None,None,3,"POLYGON ((-79.46021 43.68156, -79.46044 43.681..."
3,26005444,Woodbine Corridor (64),2480064,064,Woodbine Corridor (64),064,None,None,None,None,17545153,None,3.052518e+06,7512.966773,None,None,4,"POLYGON ((-79.31485 43.66674, -79.31660 43.666..."
4,26005443,Lawrence Park South (103),2480063,103,Lawrence Park South (103),103,None,None,None,None,17545169,None,6.211341e+06,13530.370002,None,None,5,"POLYGON ((-79.41096 43.70408, -79.41165 43.703..."


In [16]:
#SHREYA

#Income in each neighborhood
df = pd.read_csv('neighbourhood-profiles-2016.csv')
df.head()
del df['Topic']
del df['Category']
df.rename(columns = {'Characteristic':'Neighborhood'}, inplace = True)

#Extracting column for income
df1 = df.iloc[[1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053]]

#Flips rows and columns
df2 = df1.transpose()
df2.head()

print(df2.columns)
#df2.columns[""]

#df2.to_csv("Shreya.csv")

Int64Index([1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047,
            1048, 1049, 1050, 1051, 1052, 1053],
           dtype='int64')


In [33]:
#Get Latitude and Longitude from Neighbourhoods File

#Read the shape file
getCoordinates = gpd.read_file("Neighbourhoods/Neighbourhoods.shp")

#Convert to Coordinate Reference System to get Latitdue and Longitude
getCoordinates = getCoordinates.to_crs(epsg=4326)

getCoordinates.head()

#Enter data into a dataframe
neighbourhoodInformation = pd.DataFrame(getCoordinates,columns=['FIELD_7', 'FIELD_12', 'FIELD_11'])

#Rename all columns
neighbourhoodInformation.rename(columns={'FIELD_7':'Neighbourhood', 'FIELD_12':'Latitude', 'FIELD_11':'Longitude'}, inplace=True)

#Get all the values before the bracket
neighbourhoodInformation['Neighbourhood'] = neighbourhoodInformation["Neighbourhood"].astype(str)
neighbourhoodInformation['Neighbourhood'] = neighbourhoodInformation['Neighbourhood'].str.split(" \(", 1)
neighbourhoodInformation['Neighbourhood'] = neighbourhoodInformation['Neighbourhood'].str.get(0)

neighbourhoodInformation.head()
#neighbourhoodInformation.to_csv("test2.csv")

,Neighbourhood,Latitude,Longitude
0,Wychwood,43.676919,-79.425515
1,Yonge-Eglinton,43.704689,-79.403590
2,Yonge-St.Clair,43.687859,-79.397871
3,York University Heights,43.765736,-79.488883
4,Yorkdale-Glen Park,43.714672,-79.457108


In [34]:
postalCodeLongLat = pd.read_csv('PostalCode_Longitute_Latitude.csv')
#Constructing Hash

postalCodeLongLat.rename(columns={'Postal Code':'Postal Code'}, inplace=True)


postalCodeLongLat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
#Combine 

#print(crimeRateData.columns)
#print(neighbourhoodInformation.columns)
#print(finalIncome.columns)
print(postalCodeWithNeighborhoodName.columns)
print(neighbourhoodInformation.columns)

neighbourhoodInformation.head()


result = postalCodeWithNeighborhoodName.set_index('Neighbourhood').join(neighbourhoodInformation.set_index('Neighbourhood'), on='Neighbourhood')


result.head()


result.to_csv("test.csv")

Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')
Index(['Neighbourhood', 'Latitude', 'Longitude'], dtype='object')


,Postal Code,Borough,Latitude,Longitude
Neighbourhood,,,,
Parkwoods,M3A,North York,NaN,NaN
Victoria Village,M4A,North York,43.728489,-79.314874
Harbourfront,M5A,Downtown Toronto,NaN,NaN
Regent Park,M5A,Downtown Toronto,43.659992,-79.360509
Lawrence Heights,M6A,North York,NaN,NaN
